In [1]:
# Importación de librerías
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [2]:
categorias = ['vidrio', 'carton', 'papel', 'plastico']

In [3]:
# Importación del modelo
modelo = load_model('modelo.h5')

c:\Users\jeman\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\jeman\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [4]:
# Prueba con imagen
# Carga y procesamiento de la imagen
ruta_imagen = '4Clases/class_0/brown-glass1.jpg'
imagen = image.load_img(ruta_imagen, target_size=(32, 32))
imagen_arreglo = image.img_to_array(imagen)
imagen_arreglo = np.expand_dims(imagen_arreglo, axis=0)
imagen_arreglo /= 255.0

# Predicción
prediccion = modelo.predict(imagen_arreglo)
categoria = np.argmax(prediccion)

print("Probabilidades por categoría: ")
for categoria, probabilidad in enumerate(prediccion):
    print(f"{categorias[categoria]}: {probabilidad}")

print(f'\nLa categoría es: {categorias[categoria]}[{categoria}] con una probabilidad de {np.max(prediccion):.5f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Probabilidades por categoría: 
vidrio: [9.9697161e-01 3.7827151e-04 3.8661720e-04 2.2635136e-03]

La categoría es: vidrio[0] con una probabilidad de 0.99697


# Visión por computadora

In [5]:
# Crear las carpetas de clasificación si no existen
for categoria in categorias:
    if not os.path.exists(categoria):
        os.makedirs(categoria)

In [1]:
# Umbral de probabilidad máxima
umbral_probabilidad = 0.9

# Registro de imágenes
contador_imagenes = {categoria: 0 for categoria in categorias}

# Inicializar la captura de video (0 para la cámara por defecto)
camara = cv2.VideoCapture(0)

# Proceso de clasificación
while True:
    # Capturar frame por frame
    retorno, frame = camara.read()

    # Verificar si el frame fue capturado correctamente
    if not retorno:
        break

    # Preprocesar el frame
    imagen = cv2.resize(frame, (32, 32))                # Ajustar el tamaño según lo que espera tu modelo
    imagen_arreglo = np.expand_dims(imagen, axis=0)     # Expandir las dimensiones
    imagen_arreglo = imagen_arreglo / 255.0             # Normalizar la imagen si es necesario

    # Hacer la predicción
    prediccion = modelo.predict(imagen_arreglo)
    categoria = np.argmax(prediccion)
    probabilidad_maxima = np.max(prediccion)

    # Mapeo del índice a la categoría
    nombre_categoria = categorias[categoria]

    # Verificar si la probabilidad máxima supera el umbral
    if probabilidad_maxima >= umbral_probabilidad:
        # Guardar la imagen en la carpeta correspondiente
        contador_imagenes[nombre_categoria] += 1
        nombre_imagen = f"{nombre_categoria}_{contador_imagenes[nombre_categoria]}.png"
        ruta_imagen = os.path.join(nombre_categoria, nombre_imagen)
        # Guardar la imagen original (frame)
        cv2.imwrite(ruta_imagen, frame)

        # Mostrar el índice de la categoría en la imagen
        cv2.putText(frame, f'Categoría: {nombre_categoria}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Mostrar el frame con la predicción
    cv2.imshow('Predicción', frame)

    # Romper el loop con la tecla 'q' o el num 1
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

camara.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined